In [1]:
import os
from pyspark import SparkContext
sc = SparkContext(appName="search", master=os.environ['MASTER'])
import swiftclient.client

In [3]:
import sys                                                                                                                         
import itertools
import pysam
from math import sqrt                                                                                                              
from operator import add                                                                                                           
from os.path import join, isfile, dirname                                                                                          

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [4]:
config = {'user':os.environ['OS_USERNAME'], 
          'key':os.environ['OS_PASSWORD'],
          'tenant_name':os.environ['OS_TENANT_NAME'],
          'authurl':os.environ['OS_AUTH_URL']}
conn = swiftclient.client.Connection(auth_version=2, **config)

In [5]:
# List containers
(response, bucket_list) = conn.get_account()
for bucket in bucket_list:
    print bucket['name']

1000GenomesSH
GenomeData
molns_storage_04c59c75-6602-449d-bc8a-61cb382560b1
my-new-container
pyurdme_data_hdf5
test_bucket_ah
test_hadoop_over_swift
twitter_data


In [12]:
(response, obj_list) = conn.get_container('GenomeData')
for obj in obj_list:
    #if obj['name'].endswith('.bam'):
     print obj['name']
#    obj_tuple = conn.get_object(conn.get_container('1000GenomesSH'), obj['name'])
#    with open(obj['name'], 'wb') as my_Genome:
#        my_Genome.write(obj_tuple[1])

HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20120522.bam
HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20120522.bam.bai
HG00096.chrom20.ILLUMINA.bwa.GBR.low_coverage.20120522.bam.bas
HG00097.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam
HG00097.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bai
HG00097.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bas
HG00099.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam
HG00099.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bai
HG00099.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bas
HG00100.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam
HG00100.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bai
HG00100.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bas
HG00101.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam
HG00101.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bai
HG00101.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam.bas
HG00102.chrom20.ILLUMINA.bwa.GBR.low_coverage.20130415.bam
HG00102.chrom20.

In [7]:
def getCigar100M(l):
    fields = l.split('\t')
    if ( fields[5] ==  '100M'):
        return ('true')

def getBP(l):
    fields = l.split('\t')
    return (  fields[9]),abs(int(fields[8]))    

In [ ]:
# Count all individuals/region
alllines = sc.textFile("/genome/*.bam").map(lambda line: line.split('\t')[9]).map(lambda word: (word, int(1)))
allGroup = alllines.groupByKey().map(lambda x : (x[0], sum(list(x[1]))))

# Cigarstring of 100M / region
lines = sc.textFile("/genome/*.bam").filter(getCigar100M)
Cigar = lines.map(lambda line: line.split('\t')[9]).map(lambda word: (word, int(1)))
CigarGroup = Cigar.groupByKey().map(lambda x : (x[0], sum(list(x[1]))))

In [ ]:
for y in allGroup.collect(): print y

In [187]:
TotGenomes=lines.count()
print TotGenomes
for x in CigarGroup.collect():     print x
    

759002
(u'MXL', 5423)
(u'PUR', 17777)
(u'TSI', 53660)
(u'YRI', 38057)
(u'PJL', 30317)
(u'GWD', 25768)
(u'CDX', 17032)
(u'GBR', 76202)
(u'IBS', 18869)
(u'CLM', 27946)
(u'KHV', 15050)
(u'FIN', 17702)
(u'ASW', 6785)
(u'CHB', 35463)
(u'CHS', 20437)
(u'LWK', 7108)
(u'ESN', 48953)
(u'BEB', 45736)
(u'ITU', 28330)
(u'GIH', 22360)
(u'CEU', 63375)
(u'JPT', 25125)
(u'PEL', 26456)
(u'MSL', 45142)
(u'ACB', 16605)
(u'STU', 23324)


In [188]:
print(alllines.count())

2510548


In [178]:
# Average BP
lines = sc.textFile("/genome/*.bam")
BP = lines.map(getBP)

rdd1 = BP.aggregateByKey((0,0), lambda a,b: (a[0] + b,    a[1] + 1),
                                      lambda a,b: (a[0] + b[0], a[1] + b[1]))
finalResult = rdd1.mapValues(lambda v: v[0]/v[1]).collect()
print(finalResult)


[(u'MXL', 11997543), (u'PUR', 12036265), (u'TSI', 12088348), (u'YRI', 10917916), (u'PJL', 11783368), (u'GWD', 13075566), (u'CDX', 10512650), (u'GBR', 16978063), (u'IBS', 10580767), (u'CLM', 10736901), (u'KHV', 12358565), (u'FIN', 11037789), (u'ASW', 16568154), (u'CHB', 12871068), (u'CHS', 12529608), (u'LWK', 15061156), (u'ESN', 11904081), (u'BEB', 11530462), (u'ITU', 11492254), (u'GIH', 11018053), (u'CEU', 15440611), (u'JPT', 14258774), (u'PEL', 11008014), (u'MSL', 11917228), (u'ACB', 12535148), (u'STU', 11612720)]
